<a href="https://colab.research.google.com/github/Nohyeong/ML-BO_membrane_design/blob/main/Li_Bayesian_final_rej.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
!pip install rdkit
!pip install category_encoders
!pip install shap
!pip install catboost

Mounted at /gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
import os
import csv
import ast
import pytz
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from rdkit import Chem
from datetime import datetime
from rdkit.Chem import AllChem
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import category_encoders as ce
from hyperopt import hp, tpe, Trials, STATUS_OK, fmin
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from rdkit import DataStructs, Chem
from rdkit.Chem import AllChem
import shap
from sklearn.model_selection import cross_validate, ShuffleSplit, KFold
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,ARDRegression,BayesianRidge,ElasticNet,GammaRegressor,HuberRegressor
from sklearn.linear_model import Lasso, LassoLars, LinearRegression, LogisticRegression, PassiveAggressiveRegressor,Ridge,SGDRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer

import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score
from scipy.stats import linregress

def make_plot(total_data, range, tick):

  X_title = 'Real removal (%)'
  Y_title = 'Pred removal (%)'

  title_font_size = 43 # 33
  tickfont_size = 35 # 23
  ticklen = 5 # 5
  tickwidth = 2 # 2
  title_standoff = 20

  plot_width = 610 # 700
  plot_height = 500 # 500

  fig = px.scatter(total_data, x='y_true', y='y_pred', color = 'mark',
                  width=plot_width, height=plot_height, range_x=[-5, range], range_y=[-5, range]
                  #,color_continuous_scale='bluered'
                  )

  fig.update_layout(
      xaxis = dict(
          dtick=tick,
          #tickangle = 90,
          title_text = X_title,
          title_font = {"size": title_font_size},
          tickfont = dict(family='Arial', color='black', size=tickfont_size),
          title_standoff = title_standoff,
          color='black',
          ticks="inside",
          ticklen=ticklen,
          tickwidth=tickwidth,
          title_font_family="Arial"),
      yaxis = dict(
          dtick = tick,
          title_text = Y_title,
          title_font = {"size": title_font_size},
          tickfont = dict(family='Arial', color='black', size=tickfont_size),
          title_standoff = title_standoff,
          color='black',
          ticks="inside",
          ticklen=ticklen,
          tickwidth=tickwidth,
          title_font_family="Arial"))

  fig.update_layout(plot_bgcolor='rgb(256,256,256)', showlegend=True)

  fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
  fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)

  random_x = [0, 100]
  random_y0 = [0, 100]

  fig.add_trace(go.Scatter(x=random_x, y=random_y0, line=dict(color='red', width=2), marker=dict(size=1)))

  fig.update_traces(marker=dict(size=10, opacity=0.65, line=dict(width=0.7)), selector=dict(mode='markers'))

  fig.show()

def hyp_opt(objective):
  # fp_length_list = [2048]

  out_file = file_directory + 'per_model_0418.csv'
  off_connection =open(out_file, 'w')
  writer = csv.writer(off_connection)
  writer.writerow(['loss','train_loss', 'params', 'iteration'])
  off_connection.close()
  tpe_algo = tpe.suggest
  bayes_trial = Trials()

  global ITERATION
  ITERATION = 0

  tpe_algo = tpe.suggest
  bayes_trial = Trials()

  best = fmin(fn=objective, space=space, algo=tpe_algo, trials=bayes_trial, max_evals=50, rstate=np.random.default_rng(np.random.randint(0,50)))
  best['fp_length'] = int(fp_length_list[best['fp_length']])

  return best

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

class morgan_fp:
    def __init__(self, radius, length):
        self.radius = radius
        self.length = length

    def __call__(self, smiles):
        if pd.isna(smiles):
            smiles = ""

        mol = Chem.MolFromSmiles(smiles)

        if mol:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, self.length)
            npfp = np.array(list(fp.ToBitString())).astype('float32')
        else:
            npfp = np.zeros(self.length, dtype='float32')

        return npfp

def fit(params, train_data, y_train, numeric_features, salt_per):
    y_train_clean = y_train.replace([np.inf, -np.inf], np.nan).dropna()
    train_data_clean = train_data.loc[y_train_clean.index]

    fp = morgan_fp(0, 2048)
    model = XGBRegressor(max_delta_step=params['max_delta_step'], learning_rate=params['learning_rate'],
                         max_depth=params['max_depth'], min_child_weight=params['min_child_weight'],
                         subsample=params['subsample'], reg_alpha=params['reg_alpha'], gamma=params['gamma'],
                         reg_lambda=params['reg_lambda'], n_estimators=params['n_estimators'], random_state=10)
    kf = KFold(shuffle=True, random_state=10)
    t_rmse = []
    v_rmse = []

    train_data_clean.reset_index(drop=True, inplace=True)
    y_train_clean.reset_index(drop=True, inplace=True)
    en.fit(train_data_clean[categorical_features])

    for train_index, test_index in kf.split(train_data_clean):
        x_train = train_data_clean.loc[train_index].reset_index(drop=True)
        y_train_split = y_train_clean.loc[train_index].reset_index(drop=True)

        x_val = train_data_clean.loc[test_index].reset_index(drop=True)
        y_val = y_train_clean.loc[test_index].reset_index(drop=True)

        x_train = conv_data_pd(x_train, x_train, fp, en, sc, numeric_features, salt_per)
        x_val = conv_data_pd(x_val, x_train, fp, en, sc, numeric_features, salt_per)

        model.fit(x_train, y_train_split)
        t_rmse.append(np.sqrt(mean_squared_error(y_train_split, model.predict(x_train))))
        v_rmse.append(np.sqrt(mean_squared_error(y_val, model.predict(x_val))))

    return np.mean(v_rmse), np.mean(t_rmse)

def conv_data_pd(data, data_t, fp, en, scaler, numeric_features, salt_per):
    data['a1-fp'] = data['a1-smile'].apply(fp)
    data['a2-fp'] = data['a2-smile'].apply(fp)
    data['a-fp'] = [x + y for x, y in zip(data['a1-fp'], data['a2-fp'])]
    data['b-fp'] = data['b-smile'].apply(fp)

    x_a = np.array(list(data['a-fp']))
    x_b = np.array(list(data['b-fp']))

    columns_to_drop = ['Monomer A1 type', 'a1-smile', 'Monomer A2 type', 'a2-smile', 'b-smile', 'a1-fp', 'a2-fp', 'a-fp', 'b-fp']

    if salt_per == 'per':
        X_train = data_t.drop(columns=columns_to_drop, errors='ignore').copy()
        x = data.drop(columns=columns_to_drop, errors='ignore').copy()
    elif salt_per == 'rej':
        X_train = data_t.drop(columns=columns_to_drop, errors='ignore').copy()
        x = data.drop(columns=columns_to_drop, errors='ignore').copy()

    col_pd = en.transform(x[categorical_features])
    xxxx = x[numeric_features]
    num_pd = pd.DataFrame(data=xxxx, columns=numeric_features)
    xxxxx = np.concatenate((x_a, x_b), axis=1)
    fp_pd = pd.DataFrame(data=xxxxx, columns=[f'f_{i}' for i in range(2 * x_a.shape[1])])

    x_pd = pd.concat([fp_pd, col_pd, num_pd], axis=1)
    x_pd.reset_index(drop=True, inplace=True)

    return x_pd

def objective_rej(params):
    global ITERATION
    ITERATION +=1
    for name in ['max_depth', 'n_estimators','fp_length']:
        params[name] = int(params[name])
    loss, train_loss = fit(params, total_rej, y_rej, numeric_features_rej, 'rej')
    out_file = 'outfile.csv'
    off_connection = open(out_file, 'a')
    writer = csv.writer(off_connection)
    writer.writerow([loss,train_loss, params, ITERATION])
    bayes_trial = Trials()
    pickle.dump(bayes_trial, open("rej_2_xgb.p", "wb"))
    return {'loss':loss,'train_loss':train_loss, 'params': params, 'iteration':ITERATION, 'status':STATUS_OK}

def objective_per(params):
    global ITERATION
    ITERATION +=1
    for name in ['max_depth', 'n_estimators','fp_length']:
        params[name] = int(params[name])
    loss, train_loss = fit(params, total_per, y_per, numeric_features_per, 'per')
    out_file = 'outfile.csv'
    off_connection = open(out_file, 'a')
    writer = csv.writer(off_connection)
    writer.writerow([loss,train_loss, params, ITERATION])
    bayes_trial = Trials()
    pickle.dump(bayes_trial, open("per_2_xgb.p", "wb"))
    return {'loss':loss,'train_loss':train_loss, 'params': params, 'iteration':ITERATION, 'status':STATUS_OK}


def data_split(train_data_per, n_splits, test_size, random_state):
  sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=10)

  for train_index, test_index in sss.split(train_data_per):
      train_data = train_data_per.iloc[train_index]
      test = train_data_per.iloc[test_index]
      train_data.reset_index(drop=True, inplace=True)
      test.reset_index(drop=True, inplace=True)

  return train_data, test

file_directory = '/gdrive/My Drive/ML-BO_followup/'
sc = StandardScaler()

fp_length_list = [2048]

space = {'max_delta_step': hp.uniform('max_delta_step', 0, 10),
         'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
         'max_depth': hp.quniform('max_depth', 1,10,1),
         'min_child_weight': hp.uniform('min_child_weight', 1,10),
         'subsample': hp.uniform('subsample', 0.5, 1.0),
         'reg_alpha':hp.uniform('reg_alpha', 0, 10),
         'gamma':hp.uniform('gamma', 0, 10),
         'reg_lambda':hp.uniform('reg_lambda', 0, 10),
         'n_estimators': hp.quniform('n_estimators', 1, 1000, 1),
         'fp_length': hp.choice('fp_length', fp_length_list)}

In [ ]:
from sklearn.model_selection import train_test_split

# model_rej

now_utc = datetime.now(pytz.utc)
est_timezone = pytz.timezone('US/Eastern')
now_est = now_utc.astimezone(est_timezone)
date = now_est.strftime('%Y%m%d_%H:%M:%S')

title_rej = 'rej_prediction_XGBoost'

directory_rej = '/gdrive/My Drive/ML-BO_followup/prediction_result/' + date + '_' + title_rej + '/'
createFolder(directory_rej)

columns_removed = ['Pure_water_flux, LMH/bar', 'DOI', 'cation','Additive X2 type in aqueous phase',
                   'Additive X2 concentration, wt%', 'Transmembrane pressure, bar', 'anion', 'Aqueous phase pH']


df = pd.read_excel('/gdrive/My Drive/Li-pore_size/ML-BO_data_Li.xlsx', sheet_name='Datasets')
df.drop(columns_removed, axis=1, inplace=True)

df.columns = ['Monomer A1 type', 'a1-smile', 'Monomer A2 type', 'a2-smile',
              'A1/A2 ratio', 'Monomer A concentration', 'Monomer B type',
              'b-smile', 'Monomer B concentration', 'Organic solvent type',
              'Additive X1 in aqueous', 'Additive X1 concentration',
               'Additive Y type in organic  phase',
              'Additive Y concentration', 'Heat curing time',
              'Heat curing termperature',
              'salt_concentration', 'cat_val', 'cat_r', 'cat_hyd_r', 'cat_hyd_e',
              'an_val', 'an_r', 'an_hyd_r', 'an_hyd_e', 'Polymerization time',
              'Permeability', 'salt_rejection', 'Done']

df['Monomer A concentration'] = pd.to_numeric(df['Monomer A concentration'], errors='coerce')
df['salt_rejection'] = pd.to_numeric(df['salt_rejection'], errors='coerce')

categorical_features = ['Organic solvent type', 'Additive X1 in aqueous']

numeric_features_rej = ['A1/A2 ratio', 'Monomer A concentration', 'Monomer B concentration', 'Additive X1 concentration',
                        'Additive Y concentration', 'Heat curing time', 'Heat curing termperature',
                        'salt_concentration', 'cat_val', 'cat_r', 'cat_hyd_r', 'cat_hyd_e', 'an_val', 'an_r', 'an_hyd_r',
                        'an_hyd_e', 'Polymerization time']

numeric_features_per = ['A1/A2 ratio', 'Monomer A concentration', 'Monomer B concentration', 'Additive X1 concentration',
                        'Additive Y concentration', 'Heat curing time', 'Heat curing termperature',
                        'salt_concentration', 'Polymerization time']

df[numeric_features_rej] = df[numeric_features_rej].replace('-', np.nan)

df = df[df['a1-smile'] != 'xxx']
df = df[df['a2-smile'] != 'xxx']
df = df[df['b-smile'] != 'xxx']
df = df[df['Monomer A1 type'] != 'Potassium 2,5-dihydroxybenzenesulfonate']

df['a1-smile'] = df['a1-smile'].fillna('C')
df['a2-smile'] = df['a2-smile'].fillna('C')
df['b-smile'] = df['b-smile'].fillna('C')

total_df = df[df['Done']==1]
total_df.reset_index(drop=True, inplace=True)
total_df.drop(columns=['Done', 'Monomer B type'], inplace=True)

total_df['salt_rejection'] = total_df['salt_rejection'].replace('-', np.nan)
total_df['Permeability'] = total_df['Permeability'].replace('-', np.nan)
total_df['a2-smile'] = total_df['a2-smile'].replace('-', np.nan)

total_rej = total_df.dropna(subset=['salt_rejection'])
y_rej = total_rej['salt_rejection']
total_rej = total_rej.drop(columns=['Permeability', 'salt_rejection'])

X_train_rej, X_test_rej, y_train_rej, y_test_rej = train_test_split(total_rej, y_rej, test_size=0.2, random_state=42)
X_train_rej.reset_index(drop=True, inplace=True)
X_test_rej.reset_index(drop=True, inplace=True)

total_per = total_df.dropna(subset=['Permeability'])
y_per = total_per['Permeability']
total_per = total_per.drop(columns=['Permeability', 'salt_rejection'])

X_train_per, X_test_per, y_train_per, y_test_per = train_test_split(total_per, y_per, test_size=0.2, random_state=42)
X_train_per.reset_index(drop=True, inplace=True)
X_test_per.reset_index(drop=True, inplace=True)

en=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features)
en_per=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features)

train_data_rej, val_data_rej = data_split(X_train_rej, 10, 0.2, 10)
train_data_rej.reset_index(drop=True, inplace=True)
val_data_rej.reset_index(drop=True, inplace=True)

fp_rej = morgan_fp(0, 2048)
kf = KFold(shuffle=True, random_state=10)
sc_rej = PowerTransformer()

In [ ]:
params_rej = hyp_opt(objective_rej)

100%|██████████| 50/50 [1:37:10<00:00, 116.61s/trial, best loss: 11.898612640387647]


In [ ]:
y_train_rej.reset_index(drop=True, inplace=True)
y_test_rej.reset_index(drop=True, inplace=True)

In [ ]:
fp = morgan_fp(0, params_rej['fp_length'])
sc = PowerTransformer()

x_train_rej = conv_data_pd(X_train_rej, X_train_rej, fp, en, sc, numeric_features_rej, 'rej')
model_rej = XGBRegressor(max_delta_step=params_rej['max_delta_step'], learning_rate = params_rej['learning_rate'],
                    max_depth = int(params_rej['max_depth']), min_child_weight= params_rej['min_child_weight'],
                    subsample=params_rej['subsample'],reg_alpha=params_rej['reg_alpha'],gamma= params_rej['gamma'],
                    reg_lambda= params_rej['reg_lambda'],n_estimators=int(params_rej['n_estimators']), random_state=10)
model_rej.fit(x_train_rej, y_train_rej)

x_test_rej = conv_data_pd(X_test_rej, X_train_rej, fp, en, sc, numeric_features_rej, 'rej')

y_train_rej_pred = model_rej.predict(x_train_rej)
y_test_rej_pred = model_rej.predict(x_test_rej)

x_train_rej['y_true'] = y_train_rej
x_test_rej['y_true'] = y_test_rej
x_train_rej['y_pred'] = y_train_rej_pred
x_test_rej['y_pred'] = y_test_rej_pred

x_train_rej['mark'] = 'train'
x_test_rej['mark'] = 'test'
total_data_rej = pd.concat([x_train_rej, x_test_rej])

train_pred_mae = mean_absolute_error(y_train_rej, y_train_rej_pred)
train_pred_rmse = mean_squared_error(y_train_rej, y_train_rej_pred, squared = False)

pred_mae = mean_absolute_error(y_test_rej_pred, y_test_rej)
pred_rmse = mean_squared_error(y_test_rej_pred, y_test_rej, squared = False)

print(F"MAE: {np.round(pred_mae,2)}, RMSE: {np.round(pred_rmse,2)}")
print(F"Train MAE: {np.round(train_pred_mae,2)}, Train RMSE: {np.round(train_pred_rmse,2)}")

r2 = r2_score(y_test_rej_pred, y_test_rej)
print("R2: ", np.round(r2,2))
make_plot(total_data_rej, 120, 40)

In [ ]:
x_train_new = x_train_rej.drop(['y_true', 'y_pred', 'mark'], axis=1)

explainer = shap.TreeExplainer(model_rej)
shap_value_train_rej = explainer.shap_values(x_train_new)
shap.summary_plot(shap_value_train_rej, x_train_new, max_display=20, show=False)
plt.xlabel('SHAP value', fontsize=18,weight='bold')
plt.xticks(fontsize=18, weight='bold')
plt.yticks(fontsize=18, weight='bold')
plt.tight_layout()
plt.savefig('per_20.png', dpi = 400)

In [ ]:
# model_per

now_utc = datetime.now(pytz.utc)
est_timezone = pytz.timezone('US/Eastern')
now_est = now_utc.astimezone(est_timezone)
date = now_est.strftime('%Y%m%d_%H:%M:%S')

title_per = 'per_prediction_XGBoost'

directory_per = '/gdrive/My Drive/Li_Bayesian/ML-BO_followup/prediction_result/' + date + '_' + title_rej + '/'
createFolder(directory_per)

columns_removed = [' Additive Y type in organic  phase', ' Additive Y type in organic  phase', 'Pure_water_flux, LMH/bar', 'DOI', 'cation', 'anion']


df = pd.read_excel('/gdrive/My Drive/ML-BO_followup/ML-BO_data_20240430.xlsx', sheet_name='Datasets')
df.drop(columns_removed, axis=1, inplace=True)

df.columns = ['Monomer A1 type', 'Monomer A2 type', 'A1/A2 ratio',
              'Monomer A concentration', 'Monomer B type',
              'Monomer B concentration', 'Organic solvent type',
              ' Additive X1 in aqueous', 'Additive X1 concentration',
              ' Additive X2 type in aqueous phase', 'Additive X2 concentration',
              'Aqueous phase pH', 'Additive Y concentration',
              'Heat curing time', 'Heat curing termperature',
              'Transmembrane pressure', 'salt_concentration', 'cat_val', 'cat_r',
              'cat_hyd_r', 'cat_hyd_e', 'an_val', 'an_r', 'an_hyd_r', 'an_hyd_e',
              'Polymerization time', 'Permeability', 'salt_rejection',
              'Done']

In [ ]:
monomer_a = pd.read_excel('/gdrive/My Drive/ML-BO_followup/ML-BO_data_20240430.xlsx', sheet_name='monomer_a')
monomer_b = pd.read_excel('/gdrive/My Drive/ML-BO_followup/ML-BO_data_20240430.xlsx', sheet_name='monomer_b')
monomer_a = monomer_a[['Monomer', 'SMILES']]
monomer_a.dropna(inplace=True)

df = df.merge(monomer_a, how='left', left_on='Monomer A1 type', right_on='Monomer')
df.rename(columns={'SMILES': 'a1-smile'}, inplace=True)
df.drop(columns=['Monomer'], inplace=True)
df = df.merge(monomer_a, how='left', left_on='Monomer A2 type', right_on='Monomer')
df.rename(columns={'SMILES': 'a2-smile'}, inplace=True)
df.drop(columns=['Monomer'], inplace=True)
df = df.merge(monomer_b, how='left', left_on='Monomer B type', right_on='Monomer')
df.rename(columns={'SMILES': 'b-smile'}, inplace=True)
df.drop(columns=['Monomer','ID'], inplace=True)

df = df[df['a1-smile'] != 'xxx']
df = df[df['a2-smile'] != 'xxx']
df = df[df['b-smile'] != 'xxx']

# df['a1-smile'] = df['a1-smile'].fillna('C')
# df['a2-smile'] = df['a2-smile'].fillna('C')
# df['b-smile'] = df['b-smile'].fillna('C')

total_df = df[df['Done']==1]
total_df.reset_index(drop=True, inplace=True)
total_df.drop(columns=['Done', 'Monomer B type'], inplace=True)

total_per = total_df[total_df['Permeability']!='-']
y_per = total_per['Permeability']
total_per.drop(columns=['Permeability', 'salt_rejection'], inplace=True)


X_train_per, X_test_per, y_train_per, y_test_per = train_test_split(total_per, y_per, test_size=0.2, random_state=42)

X_train_per.reset_index(drop=True, inplace=True)
X_test_per.reset_index(drop=True, inplace=True)

In [ ]:
en=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features)

train_data_per, val_data_per = data_split(total_per, 10, 0.2, 10)
train_data_per.reset_index(drop=True, inplace=True)
val_data_per.reset_index(drop=True, inplace=True)

fp = morgan_fp(0, 1024)
kf = KFold(shuffle=True, random_state=10)
sc = PowerTransformer()

params_per = hyp_opt(objective_per)

In [ ]:
y_train_per.reset_index(drop=True, inplace=True)
y_test_per.reset_index(drop=True, inplace=True)

In [ ]:
fp = morgan_fp(0, params_per['fp_length'])
sc = PowerTransformer()

x_train_per = conv_data_pd(X_train_per, X_train_per, fp, en, sc, numeric_features_per, 'per')
model_per = XGBRegressor(max_delta_step=params_per['max_delta_step'], learning_rate = params_per['learning_rate'],
                    max_depth = int(params_per['max_depth']), min_child_weight= params_per['min_child_weight'],
                    subsample=params_per['subsample'],reg_alpha=params_per['reg_alpha'],gamma= params_per['gamma'],
                    reg_lambda= params_per['reg_lambda'],n_estimators=int(params_per['n_estimators']), random_state=10)
model_per.fit(x_train_per, y_train_per)

x_test_per = conv_data_pd(X_test_per, X_train_per, fp, en, sc, numeric_features_per, 'per')

y_train_per_pred = model_per.predict(x_train_per)
y_test_per_pred = model_per.predict(x_test_per)

x_train_per['y_true'] = y_train_per
x_test_per['y_true'] = y_test_per
x_train_per['y_pred'] = y_train_per_pred
x_test_per['y_pred'] = y_test_per_pred

x_train_per['mark'] = 'train'
x_test_per['mark'] = 'test'
total_data_per = pd.concat([x_train_per, x_test_per])

train_pred_mae = mean_absolute_error(y_train_per, y_train_per_pred)
train_pred_rmse = mean_squared_error(y_train_per, y_train_per_pred, squared = False)

pred_mae = mean_absolute_error(y_test_per_pred, y_test_per)
pred_rmse = mean_squared_error(y_test_per_pred, y_test_per, squared = False)

print(F"MAE: {np.round(pred_mae,2)}, RMSE: {np.round(pred_rmse,2)}")
print(F"Train MAE: {np.round(train_pred_mae,2)}, Train RMSE: {np.round(train_pred_rmse,2)}")

r2 = r2_score(y_test_per_pred, y_test_per)
print("R2: ", np.round(r2,2))
make_plot(total_data_per, 35, 10)

In [ ]:
x_train_per_new = x_train_per.drop(['y_true', 'y_pred', 'mark'], axis=1)

explainer = shap.TreeExplainer(model_per)
shap_value_train_per = explainer.shap_values(x_train_per_new)
shap.summary_plot(shap_value_train_per, x_train_per_new, max_display=20, show=False)
plt.xlabel('SHAP value', fontsize=18,weight='bold')
plt.xticks(fontsize=18, weight='bold')
plt.yticks(fontsize=18, weight='bold')
plt.tight_layout()
plt.savefig('per_20.png', dpi =400)

In [ ]:
def get_positive_fp(X_train, shap_value, fp_length):

  positive_fp_per=[]
  for i in range(fp_length):
      hh=[]
      for j in X_train[f'f_{i}'][X_train[f'f_{i}']==1].index:
          hh.append(shap_value[j, i])
      if np.sum(hh)>0:
          positive_fp_per.append(i)

  return positive_fp_per

positive_fp_per = get_positive_fp(x_train_per_new, shap_value_train_per, 1024)
positive_fp_per

In [ ]:
positive_fp_rej = get_positive_fp(x_train_new, shap_value_train_rej, 2048)
positive_fp_rej

In [ ]:
positive_fp = positive_fp_rej + positive_fp_per

In [ ]:
shap.summary_plot(shap_value_train_rej[:, positive_fp], x_train_new[[f'f_{i}' for i in positive_fp]],max_display=20, show=False)
plt.xlabel('SHAP value', fontsize=18,weight='bold')
plt.xticks(fontsize=18, weight='bold')
plt.yticks(fontsize=18, weight='bold')
plt.tight_layout()

In [ ]:
items_to_remove = [881, 935]
for item in items_to_remove:
    if item in positive_fp:
        positive_fp.remove(item)

In [ ]:
shap.summary_plot(shap_value_train_rej[:, positive_fp], x_train_new[[f'f_{i}' for i in positive_fp]],max_display=20, show=False)
plt.xlabel('SHAP value', fontsize=18,weight='bold')
plt.xticks(fontsize=18, weight='bold')
plt.yticks(fontsize=18, weight='bold')
plt.tight_layout()
plt.savefig('per_20_fp.png', dpi =400)

In [ ]:
positive_fp_rej_1=[]
for i in range(0, 2*int(params_rej['fp_length'])):
    hh=[]
    for j in x_train_rej[f'f_{i}'][x_train_rej[f'f_{i}']==1].index:
        hh.append(shap_value_train_rej[j, i])
    if np.sum(hh)>0:
        positive_fp_rej_1.append(i)

In [ ]:
positive_fp_per_1=[]
for i in range(0, 2*int(params_per['fp_length'])):
    hh=[]
    for j in x_train_per[f'f_{i}'][x_train_per[f'f_{i}']==1].index:
        hh.append(shap_value_train_per[j, i])
    if np.sum(hh)>0:
        positive_fp_per_1.append(i)

In [ ]:
positive_fp_rej_1, positive_fp_per_1

([114, 926, 935, 1114, 1171, 1873], [881, 935, 1057, 1264])

In [ ]:
def ta_index(a, b):
    a_bits = np.sum(a)
    b_bits = np.sum(b)
    h=0
    for i in range(len(a)):
        if a[i]==1 and b[i]==1:
            h+=1
    c_bits=h
    return c_bits/(a_bits+b_bits-c_bits)

In [ ]:
mom = pd.read_csv('mom_limit_rej_per_0516.csv', encoding='latin1')
mom = mom[mom['commercial']=='Y']
mom = mom[['Monomer name','a-smile', 'a-mw']]
mom.columns=['Monomer name','a-smile', 'a-mw']
mom.dropna(inplace=True)
mom.reset_index(drop=True, inplace=True)
len(mom)
mom.loc[83,'a-smile']='C'
mom.loc[83,'a-mw']=0
mom_per = mom.copy()
mom_rej = mom.copy()
mom_total = mom.copy()

In [ ]:
pfp = np.zeros(2048)
for i in positive_fp_per:
    pfp[i]=1
np.sum(pfp)

2.0

In [ ]:
rfp = np.zeros(2048)
for i in positive_fp_rej:
    rfp[i]=1
np.sum(rfp)

6.0

In [ ]:
total_fp = rfp + pfp

In [ ]:
for i in range(len(mom_total)):
    smiles = mom['a-smile'][i]
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = fp_rej(smiles)
        mom_total.loc[i, 'sim'] = ta_index(fp, total_fp)

In [ ]:
mom_total.sort_values('sim', ascending= False, inplace = True)
mom_total.reset_index(drop = True, inplace =True)
# mom_total.head(15)

In [ ]:
mom_l = mom_total[mom_total['sim']>0.1]

In [ ]:
mom_l.drop_duplicates(keep='first', inplace=True)
mom_l.reset_index(drop=True, inplace=True)
mom_l